In [3]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import os
from sklearn.utils import shuffle

In [4]:
m = np.zeros((16000,16384))

In [5]:
minr = 1000000
minc = 1000000
c =0
desired_size = (128, 128)
directory = os.fsdecode('/root/autodl-tmp/verification/ROI/')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".jpeg"): 
     #load the image, convert to grayscale and flatten the 2d image 
        img = cv2.cvtColor(cv2.imread(directory+filename), cv2.COLOR_BGR2GRAY)
#         minr = min(minr, img.shape[0])
#         minc = min(minc, img.shape[1])
        #print(c)
        img = cv2.resize(img, desired_size)
        flatten_img = np.reshape(img, (1,-1))
        m[c,:] = flatten_img
        #print(filename)
        #print(img.shape)
        #print(flatten_img.shape)
        c += 1
    else:
        continue

In [6]:
Y = np.zeros((16000, 1))
for i in range(0, 200):
    Y[i*80:(i+1)*80,:] = i+1

In [7]:
m, Y = shuffle(m, Y)

In [8]:
#80% - training
#20% - testing
x_train = m[:12800,:]
x_test = m[12800:, :]
y_train = Y[:12800, :]
y_test = Y[12800:, :]

In [9]:
#Mean normalize
mean = np.mean(x_train,0)
x_train_norm = x_train - mean[np.newaxis,:]
x_test_norm = x_test - mean[np.newaxis,:]

In [10]:
def pca(X, n_pc):
    U, S, V = np.linalg.svd(X)
    components = V[:n_pc]
    projected = U[:,:n_pc]*S[:n_pc]
    return projected, components

In [11]:
def plot_portraits(images, titles, h, w, n_row, n_col):
    plt.figure(figsize=(2.2 * n_col, 2.2 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.20)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i])
        plt.xticks(())
        plt.yticks(())

In [12]:
import scipy

In [ ]:
u,s,v = scipy.linalg.lapack.dgesvd(x_train_norm)

In [ ]:
# n_components = 50
# #P, C = pca(x_train_norm, n_pc=n_components)
# cov_matrix = np.cov(x_train_norm.T)
# print("Cov done")
# val, vec = np.linalg.eigh(cov_matrix)
# C = vec[:n_components].T
# print("PCA done")

In [ ]:
eigenpalms= C.reshape((n_components, 128, 128))
eigenpalm_titles = ["eigenface %d" % i for i in range(eigenpalms.shape[0])]
plot_portraits(eigenpalms, eigenpalm_titles, 128, 128, 4, 4) 

In [ ]:
x_train_pca = pca.transform(x_train_norm)
X_test_pca = pca.transform(x_test_norm)